In [36]:
import numpy as np
import numpy as np
import scipy.sparse as sp
import torch

import torch.nn as nn
import torch.nn.functional as fun
import torch.nn.init as init
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer

In [19]:
"""
load data from data set.
"""
def load_data():
    cat_data = np.load("facebook.npz")
    n = len(cat_data["target"])
    x = np.zeros((n, n), dtype=np.float32)
    for i in cat_data["edges"]:
        x[i[0]][i[1]] = 1
    return sp.csr_matrix(x), sp.csr_matrix(cat_data["features"], dtype=np.float32), cat_data["target"]

"""
normalize the data to 1.
"""
def normalize_adj(adjacency):
    adjacency += sp.eye(adjacency.shape[0])
    degree = np.array(adjacency.sum(1))
    d_hat = sp.diags(np.power(degree, -0.5).flatten())
    return d_hat.dot(adjacency).dot(d_hat).tocoo()

def normalize_features(features):
    return features / features.sum(1)

In [20]:
adjacency, features, labels = load_data()
encode_onehot = LabelBinarizer()
labels = encode_onehot.fit_transform(labels)

adjacency = normalize_adj(adjacency)
features = normalize_features(features)
features = torch.FloatTensor(np.array(features))
labels = torch.LongTensor(np.where(labels)[1])

print(adjacency)
print(features)
print(label)

  (0, 0)	0.5000000000000001
  (0, 18427)	0.09805806756909202
  (1, 1)	0.028571428571428567
  (1, 2812)	0.02492223931396134
  (1, 4987)	0.016903085094570332
  (1, 5228)	0.020965696734438363
  (1, 5307)	0.015240998561973751
  (1, 5755)	0.020348923188911988
  (1, 6829)	0.031943828249996996
  (1, 7136)	0.033149677206589796
  (1, 8049)	0.012135707849456652
  (1, 8533)	0.027788500718836418
  (1, 8894)	0.023218173010628604
  (1, 9934)	0.019920476822239894
  (1, 10281)	0.04517539514526256
  (1, 10379)	0.007805119495830757
  (1, 10554)	0.02238868314198225
  (1, 11557)	0.019783564706223267
  (1, 12305)	0.020498001542269693
  (1, 13737)	0.021295885499998
  (1, 14344)	0.02366905341655754
  (1, 15026)	0.02577696311132335
  (1, 15785)	0.013894250359418209
  (1, 16260)	0.03253000243161777
  (1, 16590)	0.018018749253911177
  :	:
  (22467, 5339)	0.038235955645093626
  (22467, 6181)	0.04233337566673017
  (22467, 8565)	0.03984095364447979
  (22467, 9367)	0.02703690352179376
  (22467, 9986)	0.033671751485

NameError: name 'label' is not defined

In [21]:
print(features.shape[0])

22470


In [22]:
num_nodes = features.shape[0]
train_mask = np.zeros(num_nodes, dtype=bool)
val_mask = np.zeros(num_nodes, dtype=bool)
test_mask = np.zeros(num_nodes, dtype=bool)
train_mask[:140] = True
val_mask[200:500] = True
test_mask[500:1500] = True

In [28]:
class GCNLayer(nn.Module):
    def __init__(self, input_dim, output_dim, use_bias=True):
        super(GCNLayer, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.use_bias = use_bias
        self.weight = nn.Parameter(torch.Tensor(input_dim, output_dim))
        if self.use_bias:
            self.bias = nn.Parameter(torch.Tensor(output_dim))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        init.kaiming_uniform_(self.weight)
        if self.use_bias:
            init.zeros_(self.bias)

    def forward(self, adjacency, input_feature):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        support = torch.mm(input_feature, self.weight.to(device))
        output = torch.sparse.mm(adjacency, support)
        if self.use_bias:
            output += self.bias.to(device)
        return output

    
class GCN(nn.Module):
    def __init__(self, input_dim=128):
        super(GCN, self).__init__()
        self.gcn1 = GCNLayer(input_dim, 16)
        self.gcn2 = GCNLayer(16, 4)
    
    def forward(self, adjacency, feature):
        h = fun.relu(self.gcn1(adjacency, feature))
        logits = self.gcn2(adjacency, h)
        return logits

In [47]:
learning_rate = 0.1
weight_decay = 5e-4
epochs = 200

device = "cpu"
model = GCN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)


tensor_x = features.to(device)
tensor_y = labels.to(device)
tensor_train_mask = torch.from_numpy(train_mask).to(device)
tensor_val_mask = torch.from_numpy(val_mask).to(device)
tensor_test_mask = torch.from_numpy(test_mask).to(device)
indices = torch.from_numpy(np.asarray([adjacency.row, adjacency.col]).astype('int64')).long()
values = torch.from_numpy(adjacency.data.astype(np.float32))
tensor_adjacency = torch.sparse.FloatTensor(indices, values, (22470, 22470)).to(device)


def test(mask):
    model.eval()
    with torch.no_grad():
        logits = model(tensor_adjacency, tensor_x)
        test_mask_logits = logits[mask]
        predict_y = test_mask_logits.max(1)[1]
        accuarcy = torch.eq(predict_y, tensor_y[mask]).float().mean()
    return accuarcy, test_mask_logits.cpu().numpy(), tensor_y[mask].cpu().numpy()

def train():
    loss_history = []
    val_acc_history = []
    model.train()
    train_y = tensor_y[tensor_train_mask]
    for epoch in range(epochs):
        logits = model(tensor_adjacency, tensor_x)
        train_mask_logits = logits[tensor_train_mask]
        loss = criterion(train_mask_logits, train_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_acc, _, _ = test(tensor_train_mask)
        val_acc, _, _ = test(tensor_val_mask)
        loss_history.append(loss.item())
        val_acc_history.append(val_acc.item())
        print("Epoch {:03d}: Loss {:.4f}, TrainAcc {:.4}, ValAcc {:.4f}".format(
            epoch, loss.item(), train_acc.item(), val_acc.item()))
    
    return loss_history, val_acc_history

In [48]:
train()

Epoch 000: Loss 1.3964, TrainAcc 0.4143, ValAcc 0.3200
Epoch 001: Loss 1.2400, TrainAcc 0.5571, ValAcc 0.4333
Epoch 002: Loss 1.1603, TrainAcc 0.6286, ValAcc 0.4967
Epoch 003: Loss 1.0641, TrainAcc 0.6214, ValAcc 0.4833
Epoch 004: Loss 0.9598, TrainAcc 0.65, ValAcc 0.5167
Epoch 005: Loss 0.8565, TrainAcc 0.8071, ValAcc 0.6067
Epoch 006: Loss 0.7593, TrainAcc 0.8786, ValAcc 0.6900
Epoch 007: Loss 0.6604, TrainAcc 0.8857, ValAcc 0.6733
Epoch 008: Loss 0.5695, TrainAcc 0.8929, ValAcc 0.6900
Epoch 009: Loss 0.4889, TrainAcc 0.8857, ValAcc 0.7000
Epoch 010: Loss 0.4165, TrainAcc 0.95, ValAcc 0.7167
Epoch 011: Loss 0.3514, TrainAcc 0.9571, ValAcc 0.7300
Epoch 012: Loss 0.3020, TrainAcc 0.9571, ValAcc 0.7300
Epoch 013: Loss 0.2564, TrainAcc 0.9571, ValAcc 0.7300
Epoch 014: Loss 0.2199, TrainAcc 0.9643, ValAcc 0.7333
Epoch 015: Loss 0.1890, TrainAcc 0.9714, ValAcc 0.7200
Epoch 016: Loss 0.1651, TrainAcc 0.9643, ValAcc 0.7500
Epoch 017: Loss 0.1432, TrainAcc 0.9786, ValAcc 0.7600
Epoch 018: Los

([1.3963514566421509,
  1.24004065990448,
  1.1602709293365479,
  1.0641227960586548,
  0.9598279595375061,
  0.8564662933349609,
  0.7592632174491882,
  0.6604185104370117,
  0.5695254802703857,
  0.4889271855354309,
  0.4165275990962982,
  0.3514234125614166,
  0.3019949495792389,
  0.2564278244972229,
  0.21985819935798645,
  0.1890411376953125,
  0.16505202651023865,
  0.1432422250509262,
  0.12829263508319855,
  0.11634216457605362,
  0.10494226217269897,
  0.09705748409032822,
  0.0902150422334671,
  0.08619112521409988,
  0.08187592029571533,
  0.0803593248128891,
  0.07916141301393509,
  0.07867401838302612,
  0.07922174036502838,
  0.0808560773730278,
  0.08274457603693008,
  0.08499888330698013,
  0.08741258084774017,
  0.09021323919296265,
  0.09229430556297302,
  0.09475058317184448,
  0.09671872854232788,
  0.09823983907699585,
  0.10003148019313812,
  0.10410627722740173,
  0.10882759839296341,
  0.10988621413707733,
  0.09835580736398697,
  0.09932655841112137,
  0.10015

In [49]:
from sklearn.manifold import TSNE
test_accuracy, test_data, test_labels = test(tensor_test_mask)
tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
low_dim_embs = tsne.fit_transform(test_data)
plt.title('tsne result')
plt.scatter(low_dim_embs[:,0], low_dim_embs[:,1], marker='o', c=test_labels)
plt.savefig("tsne.png")